In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Conv1D, TimeDistributed, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.losses import CategoricalCrossentropy
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import backend as K
import time
import numpy as np
import gc

model = load_model('saved_model_Stage0_LSTM/LSTM_epoch_53_Stage0.h5')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'], weighted_metrics=[])

predictions_train = np.load('Train_Dataset_For_LSTM_Stage1/predictions_train.npy')
labels_train = np.load('Train_Dataset_For_LSTM_Stage1/predictions_labels_train.npy')

predictions_val = np.load('Train_Dataset_For_LSTM_Stage1/predictions_val.npy')
labels_val = np.load('Train_Dataset_For_LSTM_Stage1/predictions_labels_val.npy')

predictions_test = np.load('Train_Dataset_For_LSTM_Stage1/predictions_test.npy')
labels_test = np.load('Train_Dataset_For_LSTM_Stage1/predictions_labels_test.npy')

data_train = []
data_train_labels = []

for i in range(1000000):
    if i + 35 > len(predictions_train):
        break
    data_train.append(predictions_train[i:i + 35, :])
    data_train_labels.append(labels_train[i:i+35, :])

print(np.shape(data_train))
print(np.shape(data_train_labels))

data_train = np.array(data_train)
data_train_labels = np.array(data_train_labels)
indices = np.arange(len(data_train))
np.random.shuffle(indices)
data_train = data_train[indices]
data_train_labels = data_train_labels[indices]

data_val = []
data_val_labels = []

for i in range(1000000):
    if i + 35 > len(predictions_val):
        break
    data_val.append(predictions_val[i:i + 35, :])
    data_val_labels.append(labels_val[i:i+35, :])

print(np.shape(data_val))
print(np.shape(data_val_labels))

data_test = []
data_test_labels = []

for i in range(1000000):
    if i + 35 > len(predictions_test):
        break
    data_test.append(predictions_test[i:i + 35, :])
    data_test_labels.append(labels_test[i:i+35, :])

print(np.shape(data_test))
print(np.shape(data_test_labels))

def evaluate_data(model, x_data, y_data, batch_size, num_segments=10):
    segment_size = len(x_data) // num_segments
    total_loss = 0
    total_acc = 0
    total_samples = 0

    cce = CategoricalCrossentropy()

    for i in range(num_segments):
        start_idx = i * segment_size
        end_idx = (i + 1) * segment_size if i < num_segments - 1 else len(x_data)

        x_segment = x_data[start_idx:end_idx]
        y_segment = y_data[start_idx:end_idx]

        preds = model.predict(x_segment, batch_size=batch_size, verbose=0)  # (batch, timesteps, num_classes)
        y_last = y_segment[:, -1, :]            # (batch, num_classes)
        preds_last = preds[:, -1, :]            # (batch, num_classes)

        # Loss
        loss = cce(y_last, preds_last).numpy()
        total_loss += loss * y_last.shape[0]

        # Accuracy
        pred_label = np.argmax(preds_last, axis=1)
        true_label = np.argmax(y_last, axis=1)
        acc = np.sum(pred_label == true_label)
        total_acc += acc
        total_samples += y_last.shape[0]

    avg_loss = total_loss / total_samples
    avg_acc = total_acc / total_samples

    return avg_loss, avg_acc

# Train the model
def AGIprogressBar(count, total,start):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)
    duration=time.time()-start
    print('\r[%s] %s%s ...%s sec' % (bar, percents, '%', duration),end=' ')

train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []
Batch = 2400
epochs = 100
segment_count = 5
st = time.time()

for ep in range(epochs):
    print(f'EP: {ep + 1}')
    segment_size = len(data_train) // segment_count
    for seg in range(segment_count):
        start_idx = seg * segment_size
        end_idx = start_idx + segment_size

        x_segment = data_train[start_idx:end_idx]
        y_segment = data_train_labels[start_idx:end_idx]

        indices = np.arange(len(x_segment))
        np.random.shuffle(indices)
        x_segment = data_train[indices]
        y_segment = data_train_labels[indices]

        for i in range(len(x_segment) // Batch):
            AGIprogressBar(i, len(x_segment) // Batch, st)
            x_batch = x_segment[i * Batch:(i + 1) * Batch]
            y_batch = y_segment[i * Batch:(i + 1) * Batch]
            model.train_on_batch(x_batch, y_batch)

    train_loss, train_acc = evaluate_data(
        model=model,
        x_data=data_train,
        y_data=data_train_labels,
        batch_size=2400,
        num_segments=5
    )

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    print(' ')
    print(f'Epoch {ep + 1} Training Loss = {train_loss}')
    print(f'Epoch {ep + 1} Training ACC = {train_acc}')

    val_loss, val_acc = evaluate_data(
        model=model,
        x_data=np.array(data_val),
        y_data=np.array(data_val_labels),
        batch_size=2400,
        num_segments=5
    )

    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f'Epoch {ep + 1} Validation Loss = {val_loss}')
    print(f'Epoch {ep + 1} Validation ACC = {val_acc}')

    os.makedirs('saved_model_Stage1_LSTM', exist_ok=True)
    model.save(f'saved_model_Stage1_LSTM/LSTM_epoch_{ep + 1}_Stage1.h5')
    print(f'Model saved at epoch {ep + 1}')

    tf.keras.backend.clear_session()
    gc.collect()
    model = load_model(f'saved_model_Stage1_LSTM/LSTM_epoch_{ep + 1}_Stage1.h5')

best_epoch_accuracy = val_accuracies.index(max(val_accuracies))
best_val_accuracy = val_accuracies[best_epoch_accuracy]
print(f"The best epoch based on validation accuracy is: {best_epoch_accuracy + 1}, with accuracy: {best_val_accuracy:.4f}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])

plt.show()

In [ ]:
val_losses[best_epoch_accuracy], val_accuracies[best_epoch_accuracy]

In [ ]:
# Plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])

plt.show()

In [ ]:
model = load_model('saved_model_Stage1_LSTM/LSTM_epoch_46_Stage1.h5')

In [ ]:
import numpy as np
test_data = np.load('Train_Dataset_For_LSTM_Stage1\predictions_test.npy')
test_data_label = np.load('Train_Dataset_For_LSTM_Stage1\predictions_labels_test.npy')

data_test = []
data_test_labels = []

for i in range(1000000000):
    if i + 35 > len(test_data):
        break
    data_test.append(test_data[i:i + 35, :])
    data_test_labels.append(test_data_label[i:i+35, :])

print(np.shape(data_test))
print(np.shape(data_test_labels))

(245211, 35, 128)
(245211, 35, 8)


In [4]:
import tensorflow as tf
test_data = tf.data.Dataset.from_tensor_slices((data_test, data_test_labels)).batch(12240).prefetch(tf.data.AUTOTUNE).cache()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

plt.rcParams['font.size'] = 7
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['text.usetex'] = False

y_test = np.array(data_test_labels)

y_true = np.argmax(y_test[:, -1, :], axis=1)

y_pred_prob = model.predict(test_data)

y_pred_last = y_pred_prob[:, -1, :]  # Shape: (None, 8)

y_pred = np.argmax(y_pred_last, axis=1)

cm = (confusion_matrix(y_true, y_pred, normalize='true'))*100
accuracy = np.trace(cm) / np.sum(cm)
print(f"Accuracy: {accuracy:.10f}")

print("Confusion Matrix:\n", cm)

# Gesture class labels
classes = ['DoublePat', 'FallDown', 'HorizontalSwipe', 'SlowUp', 'SwipeDown', 'SwipeLeft', 'SwipeRight', 'SwipeUp']

# Visualize the confusion matrix
fig, ax = plt.subplots(figsize=(3.5, 3.5))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(cmap=plt.cm.Blues, values_format='.2f', ax=ax, colorbar=False)
for text in disp.text_.ravel():
    if text.get_text():
        text.set_text(f"{float(text.get_text()):.2f}%")
plt.xticks(rotation=90, ha="right")
plt.title(r"[Exp-3] CM for LSTM: D$_1$ (Bathroom Environment)")
plt.savefig('[Exp-3] CM for LSTM_Stage1.png', dpi=300, bbox_inches='tight')
#plt.subplots_adjust(left=0.2, right=0.3, top=0.9, bottom=0.3)
plt.show()

In [ ]:
# hook Layers
hook=[]
id=[2]
for i in range(len(id)):
  hook.append(model.layers[id[i]].output)
ModelExtract = Model(inputs=model.input, outputs=hook)

In [6]:
indices = np.random.permutation(len(data_test))

data_test = np.array(data_test)
data_test_labels = np.array(data_test_labels)
x_test_shuffled = data_test[indices]
y_test_shuffled = data_test_labels[indices]

In [ ]:
predictions = ModelExtract.predict(x_test_shuffled[:20000])
labels = np.argmax(y_test_shuffled[:20000][:, -1, :], axis=1)

In [ ]:
# PCA
from sklearn.decomposition import PCA

plt.rcParams['font.size'] = 8
plt.rcParams['font.family'] = 'Times New Roman'

classes = ['DoublePat', 'FallDown', 'HorizontalSwipe', 'SlowUp', 'SwipeDown', 'SwipeLeft', 'SwipeRight', 'SwipeUp']
label_to_color = {
    'DoublePat': [1, 0, 0, 1],           # Red
    'SwipeDown': [0, 0.5, 0, 1],         # Green
    'SwipeUp': [0, 0, 1, 1],             # Blue
    'SlowUp': [1, 0.65, 0, 1],           # Orange
    'HorizontalSwipe': [1, 1, 0, 1],     # Yellow
    'SwipeLeft': [0, 0, 0, 1],           # Black
    'FallDown': [0.93, 0.51, 0.93, 1],   # Violet
    'SwipeRight': [0.5, 0, 0.5, 1],      # Purple
}

input_data = np.reshape(predictions, [len(predictions), -1])
pca = PCA(n_components=2)
pca.fit(input_data)
Dim2 = pca.transform(input_data)

fig, ax = plt.subplots(figsize=(4, 3))
for cl in range(8):
    chos = Dim2[np.argwhere(labels == cl).reshape([-1])]
    color = label_to_color[classes[cl]]
    ax.scatter(chos[:, 0], chos[:, 1], label=classes[cl], facecolors='none', edgecolors=color, marker='o', s=10)
ax.set_title(r"[Exp-3] LSTM: D$_1$, dense: " + str(predictions.shape))
plt.tight_layout()
plt.savefig('[Exp-3] PCA for LSTM_dense.png', dpi=1200, bbox_inches='tight')
plt.show()